<a href="https://colab.research.google.com/github/shubs202k/Hyperspectral-Image-Classification-using-Deep-Learning/blob/master/fine_tune_on_botswana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification')

In [2]:
from sample_extraction_V2_utils import *
import scipy.io as sio

#  Load Target Dataset - Botswana

In [3]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [4]:
data = uBotswana['Botswana']
ground_truth = gt_Botswana['Botswana_gt']

In [5]:
data.shape

(1476, 256, 145)

In [6]:
ground_truth.shape

(1476, 256)

# Distribution of Samples in target dataset

In [7]:
class_distribution = pd.DataFrame(np.unique(ground_truth, return_counts = True))
class_distribution = class_distribution.transpose()
class_distribution.columns = ['class','samples']
class_distribution

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [8]:
classes , counts = np.unique(ground_truth, return_counts = True)
classes = classes[1:] ## Not considering background
classes

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

# Load saved source model trained on Indian Pines

In [9]:
transfer_results, confusion_matrixes = transfer_learning(percentages = [50,60,70,80],
                                                        source_dataset = 'indian_pines',
                                                        target_dataset = 'botswana',
                                                        data = data,
                                                        ground_truth = ground_truth,
                                                        classes = classes,
                                                        overlap_ratios = [0.8,0.9,1],
                                                        channels = 103,
                                                        cube_size = 20,
                                                        learning_rate = 0.0001,
                                                        epochs = 100,
                                                        batch_size = 20)

Streaming output truncated to the last 5000 lines.
Epoch 00093: val_categorical_accuracy did not improve from 0.66165
Epoch 94/100
15/15 [==============================] - 0s 5ms/step - loss: 1.2961 - categorical_accuracy: 0.6208 - val_loss: 1.4941 - val_categorical_accuracy: 0.6241

Epoch 00094: val_categorical_accuracy did not improve from 0.66165
Epoch 95/100
15/15 [==============================] - 0s 5ms/step - loss: 1.2303 - categorical_accuracy: 0.5960 - val_loss: 1.5092 - val_categorical_accuracy: 0.5263

Epoch 00095: val_categorical_accuracy did not improve from 0.66165
Epoch 96/100
15/15 [==============================] - 0s 5ms/step - loss: 1.2680 - categorical_accuracy: 0.5406 - val_loss: 1.5044 - val_categorical_accuracy: 0.4962

Epoch 00096: val_categorical_accuracy did not improve from 0.66165
Epoch 97/100
15/15 [==============================] - 0s 5ms/step - loss: 1.2441 - categorical_accuracy: 0.6128 - val_loss: 1.5070 - val_categorical_accuracy: 0.5489

Epoch 00097: 

In [10]:
transfer_results

,Overlap_ratio,Training Samples,Test Samples,Training_Test_Split,Test_Accuracies
0,0.8,210,216,50,54.63
1,0.8,251,175,60,55.43
2,0.8,293,133,70,66.17
3,0.8,337,89,80,77.53
4,0.9,762,767,50,74.97
5,0.9,914,615,60,81.95
6,0.9,1065,464,70,77.37
7,0.9,1219,310,80,86.45
8,1.0,1567,1577,50,76.41
9,1.0,1880,1264,60,80.62


In [11]:
confusion_matrixes[-1]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,100.0
1,0,20,0,0,1,0,0,0,0,0,0,0,0,0,95.24
2,0,0,51,0,0,0,0,0,0,0,0,0,0,0,100.0
3,0,0,0,42,0,0,0,1,0,0,0,0,0,0,97.67
4,0,0,0,0,49,3,0,2,0,0,0,0,0,0,90.74
5,0,0,0,1,0,52,0,1,0,0,0,0,0,0,96.3
6,19,0,0,0,0,0,33,0,0,0,0,0,0,0,63.46
7,0,0,0,0,0,0,0,41,0,0,0,0,0,0,100.0
8,0,0,0,0,0,0,0,0,63,0,0,0,0,0,100.0
9,0,0,0,0,0,0,0,0,0,50,0,0,0,0,100.0
